Leung Wai Liu <br>
JPMC-SMM4H <br>
July 17, 2022 <br>
Task 2a DEV Weighted Logits Average Ensembling


In [8]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os
from scipy.special import softmax


In [9]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/dev.tsv', sep='\t')
models = ['bert-large-uncased', 'roberta-large']
# models = ['roberta-large']
n_models = len(models)

epoch_string = '../20_epochs_large_model/eval_testing/saved_eval_test_result_2a'
eval_logits_string = '../20_epochs_large_model/eval_testing/saved_eval_report_2a'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

# Loading up all of the results
best_f1 = pd.read_csv('../20_epochs_large_model/eval_testing/validation_stats/all_best_overall_f1_score.tsv', sep='\t')
best_f1



,Unnamed: 0,bert-large-uncased,roberta-large
0,0,0.669132,0.728242
1,1,0.712224,0.768425
2,2,0.672260,0.742091
3,3,0.664599,0.743203
4,4,0.693286,0.745573


In [10]:
# Retrieving all the predictions from the 
list_of_df = []
list_of_logits = []
list_of_f1_score = []
sum_of_all_f1_score = 0

for model in models: 
    specific_model_row = []
    specific_model_row_logits = []
    specific_model_row_f1 = []

    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_eval_test_result.tsv'
        to_read_eval_string = eval_logits_string + '/' + model + '/' + str(rnd) + '/eval_logits.tsv'

        # Reading the best f1 scores 
        specific_f1_score = best_f1.at[rnd, model]
        specific_model_row_f1.append(specific_f1_score)
        sum_of_all_f1_score += specific_f1_score
        
        # Reading each particular model df
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)

        # Reading each particular model's eval logits
        particular_model_eval_logits = pd.read_csv(to_read_eval_string, sep='\t')
        particular_model_eval_logits = particular_model_eval_logits.drop(columns = 'Unnamed: 0')
        particular_model_eval_logits = list(particular_model_eval_logits.itertuples(index=False, name=None))
        particular_model_eval_logits = softmax(particular_model_eval_logits, axis = 1)
        # particular_model_eval_logits = softmax(particular_model_eval_logits)
        specific_model_row_logits.append(particular_model_eval_logits)
    
    list_of_df.append(specific_model_row)
    list_of_logits.append(specific_model_row_logits)
    list_of_f1_score.append(specific_model_row_f1)


In [11]:
# TO VERIFY
print(list_of_logits)

[[array([[1.39619023e-02, 9.84701662e-01, 1.33643535e-03],
       [4.03744160e-03, 6.19660528e-03, 9.89765953e-01],
       [3.52047876e-02, 8.28822429e-01, 1.35972783e-01],
       ...,
       [9.97765708e-01, 4.76933171e-04, 1.75735875e-03],
       [3.59315890e-03, 9.81999773e-01, 1.44070681e-02],
       [1.75045359e-03, 9.58069720e-01, 4.01798267e-02]]), array([[3.62230323e-03, 9.95709552e-01, 6.68144648e-04],
       [3.71603840e-03, 1.97645252e-03, 9.94307509e-01],
       [1.23201767e-03, 8.17034110e-01, 1.81733872e-01],
       ...,
       [9.98291449e-01, 9.41226485e-04, 7.67324644e-04],
       [1.01684279e-03, 9.92233518e-01, 6.74963902e-03],
       [4.76174693e-05, 9.99593280e-01, 3.59102838e-04]]), array([[8.61307411e-03, 9.90329214e-01, 1.05771215e-03],
       [2.12139221e-03, 4.08072942e-03, 9.93797878e-01],
       [2.56642864e-01, 6.38857656e-01, 1.04499480e-01],
       ...,
       [9.70864719e-01, 2.79974332e-02, 1.13784789e-03],
       [8.53408143e-04, 9.97438945e-01, 1.7076

In [12]:
list_of_f1_score

[[0.6691320036128283,
  0.7122239067365979,
  0.6722600568055085,
  0.6645994604968366,
  0.6932855215649601],
 [0.7282416924157605,
  0.7684248948494871,
  0.7420911050352187,
  0.743203289233602,
  0.745572928575517]]

In [13]:
# TAKING THE WEIGHTED AVERAGE OF DATA

weighted_avg_original_tweet_id_list = []
weighted_avg_original_sentence_list = []
weighted_avg_original_claim_list = []
weighted_avg_original_label_list = []
weighted_avg_original_numbered_label_list = []

weighted_avg_predicted_number_results = []
weighted_avg_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['id']
    original_sentence = row['Tweet']
    original_claim = row['Claim']
    original_label = row['Stance']
    
    # transferring the labels over to final list
    weighted_avg_original_tweet_id_list.append(original_tweet_id)
    weighted_avg_original_sentence_list.append(original_sentence)
    weighted_avg_original_claim_list.append(original_claim)
    weighted_avg_original_label_list.append(original_label)
    
    specific_row_value = 0.0
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):

            row_logits = list_of_logits[model_num][rnd_num][index]

            row_logits_value = ((row_logits[0] * 0) + (row_logits[1] * 1) + (row_logits[2] * 2))
            prediction = row_logits_value * list_of_f1_score[model_num][rnd_num]
            print(prediction)
            specific_row_value += prediction

    specific_row_value = specific_row_value / sum_of_all_f1_score
    specific_row_result = int(round(specific_row_value))
    weighted_avg_predicted_results.append(ids_to_labels[specific_row_result])
    

0.6606838996316933
0.7101198843678145
0.667180888745016
0.6638219381115228
0.6933760795203558
0.7319856926057373
0.7687757490063767
0.7387872977041813
0.7911137875212841
0.7466071092203121
1.3287144975453575
1.417746833966874
1.3389245477282898
1.3275287396392694
1.3855839971182025
1.1713786429673039
1.535994897649259
1.4586691023227012
1.2951355557661468
1.4703058949333272
0.7365590944638163
0.8407816426210202
0.5699801365737442
1.3262150836837128
0.3399719801442623
1.2580494331090537
1.3211776027211402
1.4303767100396871
0.46932284779159095
0.7558166000853956
0.013784901364189736
0.6177282998798108
0.03262000091023826
0.6533025733972395
0.008300990956420201
0.7636587536366165
0.7703542419868494
0.029898342886821985
0.7355487225696143
0.6679112757223682
0.9452402531684224
0.9487361984864427
1.340144903204055
0.6650830020591162
0.9976357680955925
1.3243995284558563
0.7733902590591882
1.365340972608717
1.2485478321927657
1.2271536834104744
0.020635724425611172
0.022025764175700915
0.002

In [14]:
# Calculating sklearn metrics

weighted_avg_original_numbered_label_list = [labels_to_ids[label] for label in weighted_avg_original_label_list]
weighted_avg_predicted_number_results = [labels_to_ids[label] for label in weighted_avg_predicted_results]

num_overall_prediction_data = pd.DataFrame(zip(weighted_avg_original_tweet_id_list, weighted_avg_original_sentence_list, weighted_avg_original_claim_list, weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results), columns=['tweet_id', 'text', 'Claim', 'Orig', 'Stance'])

fm_f1_score, fm_precision, fm_recall, saho_f1_score, saho_precision, saho_recall, sc_f1_score, sc_precision, sc_recall = calculate_overall_performance_metrics(num_overall_prediction_data)

accuracy_result = accuracy_score(weighted_avg_original_numbered_label_list, weighted_avg_predicted_number_results)

net_f1 = (1.0/3.0) * (fm_f1_score + saho_f1_score + sc_f1_score)

print("ACCURACY:", accuracy_result)
print("F1:", net_f1)

# Saving results to file
os.makedirs('../20_epochs_large_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble', exist_ok=True)

with open('../20_epochs_large_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/weighted_avg_ensemble_valid_stats.txt', 'w') as file:
        file.write("Accuracy: " + str(accuracy_result) + "\n")
        file.write("Net F1: " + str(net_f1) + "\n")
        file.write("Ind F1 Score: " + str(fm_f1_score) + " , " + str(saho_f1_score) + " , " + str(sc_f1_score) + "\n")
        file.write("Ind Precision Score: " + str(fm_precision) + " , " + str(saho_precision) + " , " + str(sc_precision) + "\n")
        file.write("Ind Recall Score: " + str(fm_recall) + " , " + str(saho_recall) + " , " + str(sc_recall) + "\n")


Running performance metrics
Finished running performance metrics
ACCURACY: 0.77
F1: 0.7305715709339067
